#### https://techblog-history-younghunjo1.tistory.com/374

# 1. Computational Graph
- 노드(Node)와 간선(Edge)로 계산을 표현할 수 있는 그래프
- 사과 박스(Box) -> 사과 갯수(200원), 소비세(220원) -> 내야할돈(money)
    - 이와 같이 왼쪽에서 오른쪽으로 진행하는 단계를 순전파(Forward Propagation)이라고 한다.
    - 역전파(Back Propagation)는 이 반대방향인 오른쪽에서 왼쪽으로 진행하는 단계를 의미한다.

# 2. Local Computation
- 국소적(Local) : 자신과 직접 관계된 작은 범위
- 국소적 계산(Local Computation) : 자신과 직접 관계된 작은 범위내에서 이루어지는 계산
    - 각각 자신의 노드에서 이루어지는 연산은 자기 노드 이전 노드까지 어떤 복잡한 계산과정을 하던 말던 자신의 노드에 있는 연산 계산만 신경쓰면 된다.
- 이러한 특징 때문에, 국소적 계산은 복잡한 계산을 단순화하는 장점이 있다. 또한 계산 결과를 캐싱 기능처럼 모두 보관할 수 있다.

# 3. Back Propagation - Derivative
- 미분(derivative)이란 어떤 함수의 정의역 속 각 점에서 함숫값의 변화량과 독립 변숫값의 변화량 비의 극한 혹은 극한들로 치역이 구성되는 함수이다.
    - 어떤 함수의 미분 계수 또는 순간 변화율을 구하는 것을 의미한다.
- 역전파 수행 시, 순전파를 수행했을 때의 입력으로 들어온 값에 대해 입력 이후 계산 **결과값이 얼마나 변화하는지에 대한 값, 바로 '미분'값을 전달**한다.
    - 내야할돈(220원) -> 소비세(1.1) -> 사과갯수 (2.2) -> 사과박스
    - 결과적으로 2.2의 값이 사과박스 가격에 대한 내야할 돈의 이분값이다. "사과박스 가격이 아주 조금 변화했을 때, 내야할 돈이 기존보다 얼마나 변화했는가?"를 의미한다. 즉, 변화량의 계산
- 역전파는 중간중간에 존재하는 모든 연산 과정에 대해 미분(i.e. 국소적 미분)을 수행해서 최종적으로 '사과 박스 가격'에 대한 내야할 돈의 최종 미분 값을 결정해준다.

# 4. Chain Rule
- 그렇다면 연속적으로 연산 노드 마다 미분 값을 어떻게 구해줄까?
- **연쇄법칙을 통해서 역전파 시 국소적인 미분을 전달**할 수 있다.
    - 이전 노드에서 흘러나온 계산 결과값을 E로 봤을 때, f(x)에 대한 국소적 미분 값을 구하기 위해서 x에 대한 y값의 미분값을 구해주고 이를 E와 곱해서 다음 노드로 넘겨준다.
## 국소적 미분을 활용하는 연쇄법칙
- 연쇄 법칙은 합성함수의 미분을 의미한다. 합성함수의 미분을 구하기 위해서는 합성함수를 구성하는 각 함수의 비분의 곱으로 나타낼 수 있다.
    - 이를 펼치면 편미분의 형태가 나온다.

# 5. Apple Box Example
- d is derivative

In [4]:
import numpy as np

class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x * y

    def backward(self, d_out):
        # Forward Propagation 시, 입력을 서로 바꾸어서 곱해준다.
        dx = d_out * self.y
        dy = d_out * self.x
        
        return dx, dy

In [8]:
apple_box = 100
apple_box_num = 2
tax = 1.1

In [9]:
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

In [17]:
# 1. Front Propagation
# x - 사과 박스 개당 가격, y - 사과 박스 갯수
apple_box_price = mul_apple_layer.forward(apple_box, apple_box_num)
# x - 사과 박스 개당 가격 X 사과 박스 갯수 출력값, y - 소비세
price = mul_tax_layer.forward(apple_box_price, tax)
print("After Front Propagation, Price : {}".format(price))

# 2. Back Propagation
d_price = 1 
# x - 
d_apple_box_price, d_tax = mul_tax_layer.backward(d_price)
d_apple_box, d_apple_box_num = mul_apple_layer.backward(d_apple_box_price)
print("After Back Propagation")
print("사과 박스 가격 : {}".format(d_apple_box))
print("사과 박스 갯수 : {}".format(d_apple_box_num))
print("소비자세 : {}".format(d_tax))

After Front Propagation, Price : 220.00000000000003
After Back Propagation
사과 박스 가격 : 2.2
사과 박스 갯수 : 110.00000000000001
소비자세 : 200


# 6. Add Layer

In [18]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        return x + y

    def backward(self, d_out):
        dx = d_out * 1
        dy = d_out * 1
        return dx, dy

In [22]:
x = 2
y = 3

add_layer = AddLayer()
result = add_layer.forward(x, y)
print(result)

d_out = 1
dx, dy = add_layer.backward(d_out)
print(dx, dy)

5
1 1


# 7. Multiply Layer

In [24]:
class MulLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

In [27]:
x = 2
y = 3

mul_layer = MulLayer()
result = mul_layer.forward(x, y)
print(result)

d_out = 1
dx, dy = mul_layer.backward(d_out)
print(dx, dy)

6
3 2


# 8. Add + Mul Layer

In [28]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

In [32]:
# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# Forward Propagation
apple_price = mul_apple_layer.forward(apple, apple_num) # (1)
orange_price = mul_orange_layer.forward(orange, orange_num) # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price) # (3)
price = mul_tax_layer.forward(all_price, tax) # (4)

# Back Propagation
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice) # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price) # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price) # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price) # (1)

print("price:",price)
print("dApple:",dapple)
print("dApple_num",dapple_num)
print("dOrange:",dorange)
print("dOrange_num",dorange_num)
print("dTax",dtax)

price: 715.0000000000001
dApple: 2.2
dApple_num 110.00000000000001
dOrange: 3.3000000000000003
dOrange_num 165.0
dTax 650
